In [ ]:
import torch
import pandas as pd
import os

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

import io

import ipywidgets as widgets
from PIL import Image
import io

import plotly.graph_objects as go
import plotly.express as px


import sys
sys.path.append("/mnt/raid/C1_ML_Analysis/source/us-famli-pl/src/")
from nets.classification import EffnetV2sTD, EffnetV2sTDOEMD
from loaders.ultrasound_dataset import USAnnotatedBlindSweepDataModule

from tqdm.notebook import tqdm
import SimpleITK as sitk

In [ ]:

def _frame_to_png_bytes(frame2d: np.ndarray) -> bytes:
    """Convert a (C,H,W) frame (any numeric dtype) to PNG bytes for ipywidgets.Image."""
    f = np.asarray(frame2d)
    # Handle NaNs/infs safely
    f = np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0)

    # Normalize to uint8
    fmin = float(f.min())
    fmax = float(f.max())
    if fmax > fmin:
        u8 = ((f - fmin) / (fmax - fmin) * 255.0).astype(np.uint8)
    else:
        u8 = np.zeros_like(f, dtype=np.uint8)

    img = Image.fromarray(u8, mode="RGB")  # RGB
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return buf.getvalue()


def visualize_sequence(
    images: np.ndarray,
    scores: np.ndarray,
    scores_pred: np.ndarray,
    *,
    fps: float = 10.0,
    title: str = "Sequence",
    show_filename: bool = False,
    filenames=None,
):
    """
    Jupyter-stable interactive viewer:
      - Left: grayscale frame shown via ipywidgets.Image
      - Right: Plotly line plots for GT + Pred with moving markers + vertical line
      - Controls: Play + Slider

    Parameters
    ----------
    images : np.ndarray
        Shape (T,H,W) or (C,T,H,W) or (T,H,W,C) supported if you tweak below.
        (Your code uses a helper _frame_to_png_bytes(frame) that expects a single frame.)
    scores : array-like
        Shape (T,), GT score per frame.
    scores_pred : array-like
        Shape (T,), predicted score per frame.
    """
    images = np.asarray(images)
    if images.ndim != 4:
        raise ValueError(f"`images` must have shape (T,H,W,C) (per your current code). Got {images.shape}")

    # Your original code says (C,T,H,W) but then unpacks as (T,H,W,C).
    # Keeping your current behavior: images.shape == (T,H,W,C)
    T, H, W, C = images.shape

    scores_np = np.asarray(scores).reshape(-1)
    if scores_np.shape[0] != T:
        raise ValueError(f"`scores` length must match T={T}. Got {scores_np.shape[0]}")

    scores_pred_np = np.asarray(scores_pred).reshape(-1)
    if scores_pred_np.shape[0] != T:
        raise ValueError(f"`scores_pred` length must match T={T}. Got {scores_pred_np.shape[0]}")

    if show_filename:
        if filenames is None:
            raise ValueError("show_filename=True requires `filenames=`")
        filenames = np.asarray(filenames).reshape(-1)
        if filenames.shape[0] != T:
            raise ValueError(f"`filenames` length must match T={T}. Got {filenames.shape[0]}")

    # --- Widgets ---
    slider = widgets.IntSlider(value=0, min=0, max=T - 1, step=1, description="Frame", continuous_update=True)
    play = widgets.Play(
        value=0, min=0, max=T - 1, step=1,
        interval=int(1000 / max(fps, 1e-6)),
        description="Play",
    )
    widgets.jslink((play, "value"), (slider, "value"))

    # Image display (PNG bytes)
    img_w = widgets.Image(value=_frame_to_png_bytes(images[0]), format="png")

    # Metadata panel
    meta = widgets.HTML()
    def _meta_html(i: int) -> str:
        base = (
            f"<b>frame:</b> {i}"
            f"<br><b>score (gt):</b> {scores_np[i]:.3f}"
            f"<br><b>score (pred):</b> {scores_pred_np[i]:.3f}"
        )
        if show_filename:
            base += f"<br><b>file:</b> {filenames[i]}"
        return base

    meta.value = _meta_html(0)

    # Plotly FigureWidget
    x = np.arange(T)
    fig = go.FigureWidget()

    # Lines
    fig.add_scatter(x=x, y=scores_np,      mode="lines", name="score (gt)")
    fig.add_scatter(x=x, y=scores_pred_np, mode="lines", name="score (pred)")

    # Current markers (one for each series)
    fig.add_scatter(x=[0], y=[float(scores_np[0])],      mode="markers", name="current (gt)")
    fig.add_scatter(x=[0], y=[float(scores_pred_np[0])], mode="markers", name="current (pred)")

    # Add max predicted score line
    max_pred_idx = int(np.argmax(scores_pred_np))
    fig.add_scatter(
        x=[max_pred_idx],
        y=[float(scores_pred_np[max_pred_idx])],
        mode="markers",
        name="max score (pred)",
        marker=dict(color="green", size=10, symbol="x"),
    )

    # y-range for the vertical line: cover both series
    y_min = float(np.min([scores_np.min(), scores_pred_np.min()]))
    y_max = float(np.max([scores_np.max(), scores_pred_np.max()]))

    fig.update_layout(
        title=title,
        xaxis_title="Frame",
        yaxis_title="Score",
        margin=dict(l=40, r=10, t=40, b=40),
        shapes=[
            dict(
                type="line",
                x0=0, x1=0,
                y0=y_min, y1=y_max,
                xref="x", yref="y",
                line=dict(width=2, dash="dash"),
            )
        ],
    )

    def _update(i: int):
        # Update image
        img_w.value = _frame_to_png_bytes(images[i])

        # Update markers + vertical line
        with fig.batch_update():
            # marker traces are indices 2 and 3
            fig.data[2].x = (i,)
            fig.data[2].y = (float(scores_np[i]),)

            fig.data[3].x = (i,)
            fig.data[3].y = (float(scores_pred_np[i]),)

            # keep line spanning both series ranges (or recompute if you want dynamic)
            fig.layout.shapes[0].update(x0=i, x1=i, y0=y_min, y1=y_max)

        # Update metadata
        meta.value = _meta_html(i)

    slider.observe(lambda ch: _update(ch["new"]), names="value")

    controls = widgets.HBox([play, slider])
    left = widgets.VBox([meta, img_w])
    right = widgets.VBox([fig])
    ui = widgets.VBox([controls, widgets.HBox([left, right])])

    return ui


In [ ]:
mount_point = '/mnt/raid/C1_ML_Analysis'

dev_id = 0
device = torch.device(f'cuda:{dev_id}' if torch.cuda.is_available() else 'cpu')

# model_fn = os.path.join(mount_point, 'train_output/classification/EffnetV2sTD/', 'v0.1/epoch=20-val_loss=0.832.ckpt')
# model = EffnetV2sTD.load_from_checkpoint(model_fn, map_location=device)


model_fn = os.path.join(mount_point, 'train_output/classification/EffnetV2sTDOEMD/', 'v0.1/epoch=29-val_select=0.227.ckpt')
model = EffnetV2sTDOEMD.load_from_checkpoint(model_fn, map_location=device)


model.to(device)
model.eval()

In [ ]:
model.hparams

In [ ]:
dm = USAnnotatedBlindSweepDataModule(**model.hparams)
dm.setup()

In [ ]:
def run_test(dm, model):

    dirname = os.path.dirname(model_fn)
    output_csv = os.path.join(dirname, 'predictions.csv')

    if os.path.exists(output_csv):
        print(f"Predictions file {output_csv} already exists. Loading it.")
        df = pd.read_csv(output_csv)
        return df
    
    test_dl = dm.test_dataloader()
    test_ds = dm.test_ds

    df_scores = []
    with torch.no_grad():
        for idx, X_d in tqdm(enumerate(test_dl), total=len(test_dl)):
            file_path = test_ds.df.iloc[idx]['file_path']
            logits = model(X_d['img'].permute(0, 2, 1, 3, 4).to(device))  # [B, N, C]
            probs = torch.softmax(logits, dim=-1)
            levels = torch.tensor([0.0, 0.25, 0.5, 0.75, 1.0], device=probs.device)
            score = (probs * levels).sum(dim=-1)  # [B, N]

            df = pd.DataFrame({
                'frame_index': list(np.arange(X_d['img'].shape[2])),
                'file_path': file_path,
                'score_pred': score.squeeze().cpu().numpy(),
                'score': X_d['scalar'].squeeze().cpu().numpy(),
            })
            df_scores.append(df)    
    df = pd.concat(df_scores, ignore_index=True)

    df.to_csv(output_csv, index=False)
    print(f"Saved predictions to {output_csv}")
    return df

df_test_pred = run_test(dm, model)

In [ ]:
df_test_pred['error'] = df_test_pred['score_pred'] - df_test_pred['score']
df_test_pred['abs_error'] = df_test_pred['error'].abs()

In [ ]:
df_test_pred['abs_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
bins = np.array([0.0, 0.25, 0.5, 0.75, 1.0])

def to_class(x):
    return np.argmin(np.abs(bins - x))

df_test_pred["y_true"] = df_test_pred.score.apply(to_class)
df_test_pred["y_pred"] = df_test_pred.score_pred.apply(to_class)

In [ ]:
import itertools

cm = confusion_matrix(df_test_pred['y_true'], df_test_pred['y_pred'])
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(8,6))
plt.imshow(cm_norm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(bins))

fmt = '.3f' 
thresh = cm_norm.max() / 2.
for i, j in itertools.product(range(cm_norm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm_norm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm_norm[i, j] > thresh else "black")

plt.xticks(tick_marks, [f"{b:.2f}" for b in bins])
plt.yticks(tick_marks, [f"{b:.2f}" for b in bins])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()






In [ ]:
report = classification_report(df_test_pred['y_true'], df_test_pred['y_pred'], target_names=['0.0', '0.25', '0.5', '0.75', '1.0'])
print(report)

In [ ]:
m = df_test_pred.score == 1.0
plt.hist(df_test_pred.score_pred[m], bins=30)
plt.axvline(1.0, color="r")
plt.title("Predictions for true high_measurable (1.0)")
plt.show()

In [ ]:
for t in [0.7, 0.8, 0.9]:
    tp = np.mean(df_test_pred.score_pred[df_test_pred.score == 1.0] >= t)
    print(f"P(score_pred >= {t} | true=1.0) = {tp:.3f}")


In [ ]:
for t in [0.5, 0.9]:
    tp = np.mean(df_test_pred.score_pred[df_test_pred.score == 0.0] >= t)
    print(f"P(score_pred >= {t} | true=0.0) = {tp:.3f}")

In [ ]:
df_seq = (
    df_test_pred.groupby("file_path")
      .agg(score_max=("score","max"), pred_max=("score_pred","max"))
      .reset_index()
)


In [ ]:
px.violin(df_seq, y="pred_max", x="score_max", box=True, points="all", title=f"Predicted max score vs GT max score per sequence")

In [ ]:
px.violin(df_test_pred, y="score_pred", x="score", box=True, points="outliers", title=f"Predicted score vs GT score per frame")

In [ ]:
for s in [0, 0.25, 0.5, 0.75, 1.0]:
    print(f"Score: {s}")
    print(df_test_pred.query(f'score == {s}')['score_pred'].describe(percentiles=[.01, .1, .25, 0.5, 0.75, 0.9, 0.95, 0.99]))

In [ ]:
test_ds = dm.test_ds
df_frames = test_ds.df_frames
df = test_ds.df

In [ ]:
list(df_seq.query('score_max == 0.25 and pred_max > 0.8')['file_path'].drop_duplicates())

In [ ]:
idx = 2 
file_paths = list(df_seq.query('score_max <= 0.25 and pred_max > 0.7')['file_path'].drop_duplicates())
print(len(file_paths))
file_path = file_paths[idx]

# idx = 0
# file_paths = list(df_seq.query('score_max >= 0.75 and pred_max <= 0.25')['file_path'].drop_duplicates())
# print(file_paths)
# file_path = file_paths[idx]

print(file_path)
q = df_test_pred.query('file_path == @file_path')
score, score_pred = q['score'], q['score_pred']

img = sitk.ReadImage(os.path.join(mount_point, file_path))
img_np = sitk.GetArrayFromImage(img)  # [T,H,W]

if img.GetNumberOfComponentsPerPixel() == 1:
    img_np = np.expand_dims(img_np, -1).repeat(3, axis=-1)

visualize_sequence(img_np, score, score_pred, title=f"{'/'.join(file_path.split('/')[-2:])}")

In [ ]:
score

In [ ]:
df_test_pred

In [ ]:

def run_example():

    file_path = df_frames.query('annotation_label == "high_measurable" or annotation_label == "low_measurable" or annotation_label == "high_visible" and tag != "AC"')['file_path'].drop_duplicates().sample(n=1).values[0]
    
    print(file_path)
    idx = df.query(f'file_path == "{file_path}"').index[0] 
    X_d = test_ds[idx]

    with torch.no_grad():
        logits = model(X_d['img'].unsqueeze(0).permute(0, 2, 1, 3, 4).to(device))
        probs = torch.softmax(logits, dim=-1)
        levels = torch.tensor([0.0, 0.25, 0.5, 0.75, 1.0], device=probs.device)
        score = (probs * levels).sum(dim=-1)  # [B, N]


    visualize_sequence(X_d['img'].permute(1,2,3,0), X_d['scalar'], score.squeeze(0).cpu().numpy(), title=f"{'/'.join(file_path.split('/')[-2:])}")

In [ ]:
w_bins = torch.arange(1, 6, dtype=torch.float32) 
w_bins = w_bins / w_bins.mean()

In [ ]:
w_bins

In [ ]:
c = np.array([369458, 92519, 15579, 7674, 5970])
w = 1.0 / c
w = w / w.mean()
w


In [ ]:
wb = np.array([0.2,0.4,0.6,1.0,2.0])
wb = wb / wb.mean()
wb